In [101]:
import spacy
import pandas as pd
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from collections import OrderedDict

In [103]:
nlp1 = spacy.load("pl_document_ner/output/model-best")

In [104]:
df =pd.read_csv("data/processed_dataset_12_11_2021.csv")
df

,Unnamed: 0,categories,text,text_full,main_category,label_mid,label_high
0,0,114 Rzeczowe,['czy wspólnota mieszkaniowa może podjąć uchwa...,czy wspólnota mieszkaniowa może podjąć uchwałę...,rzeczowe,rzeczowe,prawo cywilne
1,1,114 Rzeczowe,['czy wspólnota mieszkaniowa może odzyskać pom...,czy wspólnota mieszkaniowa może odzyskać pomie...,rzeczowe,rzeczowe,prawo cywilne
2,3,15 Prawo ochrony danych osobowych,['czy uprawnienie mieszkańców wspólnoty mieszk...,czy uprawnienie mieszkańców wspólnoty mieszkan...,prawo ochrony danych osobowych,prawo administracyjne materialne,prawo administracyjne
3,4,"20 Prawo budowlane, 114 Rzeczowe",['czy budowa windy dla osób niepełnosprawnych ...,czy budowa windy dla osób niepełnosprawnych mo...,prawo budowlane,prawo administracyjne materialne,prawo administracyjne
4,5,114 Rzeczowe,['jakie czynności powinna podjąć wspólnota mie...,jakie czynności powinna podjąć wspólnota miesz...,rzeczowe,rzeczowe,prawo cywilne
...,...,...,...,...,...,...,...
2716,4030,77 Ustawa o prawach pacjenta i Rzeczniku Praw ...,['czy dokumentacja medyczna powinna zostać wyd...,czy dokumentacja medyczna powinna zostać wydan...,ustawa o prawach pacjenta i rzeczniku praw pac...,ustawa o prawach pacjenta i rzeczniku praw pac...,prawo medyczne
2717,4031,"59 Ustawa Prawo farmaceutyczne, 197 Inne - pra...","['czy apteka może sprzedawać wodę mineralną?',...",czy apteka może sprzedawać wodę mineralną czy ...,ustawa prawo farmaceutyczne,ustawa prawo farmaceutyczne,prawo farmaceutyczne
2718,4032,"15 Prawo ochrony danych osobowych, 69 Prawo ka...",['co w przedmiotowej sprawie może zrobić klien...,co w przedmiotowej sprawie może zrobić klientka,prawo ochrony danych osobowych,prawo administracyjne materialne,prawo administracyjne
2719,4033,79 Ustawa o zawodach lekarza i lekarza dentyst...,['czy lekarz stażysta może przeprowadzać kwali...,czy lekarz stażysta może przeprowadzać kwalifi...,ustawa o zawodach lekarza i lekarza dentysty,ustawa o zawodach lekarza i lekarza dentysty,prawo medyczne


In [180]:
def classifyText(text):
    doc = nlp1(text)
    legalAffairs = OrderedDict()
    for token in doc:
        if token.ent_type_ != '':
            if token.ent_type_ not in legalAffairs:
                legalAffairs[token.ent_type_] = 0
            legalAffairs[token.ent_type_] += 1
    legalAffairs = OrderedDict(sorted(legalAffairs.items(),key = lambda key: key[1], reverse=True))
    if len(legalAffairs) == 0:
        return 'UNKNOWN'
    # print(legalAffairs)
    return str(next(iter(legalAffairs)))


In [175]:
correctClassifications = 0
labelTags = {'administrative_law':'prawo administracyjne','civil_law':'prawo cywilne','prawo_medyczne':'medical_law','tax_law':'prawo podatkowe','criminal_law':'prawo karne','pharmaceutical_law':'prawo farmaceutyczne', 'labor_law':'prawo pracy','international_law':'prawo miädzynarodowe','constitutional_law':'prawo konstytucyjne'}
for i in range(len(df)):
    label= df['label_high'][i]
    classified = 'UNKNOWN'
    labelTag = classifyText(df['text'][i])
    if labelTag in labelTags.keys():
        classified = labelTags[labelTag]
    print('{}/{} label: {}, classified: {} | CORRECT?: {}'.format(i,len(df),label,classified ,label==classified))
    if label == classified:
        correctClassifications += 1
print('Overall accuracy: {}%'.format(correctClassifications/len(df)))

0/2721 label: prawo cywilne, classified: UNKNOWN | CORRECT?: False
OrderedDict([('criminal_law', 1)])
1/2721 label: prawo cywilne, classified: prawo karne | CORRECT?: False
2/2721 label: prawo administracyjne, classified: UNKNOWN | CORRECT?: False
OrderedDict([('pharmaceutical_law', 1)])
3/2721 label: prawo administracyjne, classified: prawo farmaceutyczne | CORRECT?: False
4/2721 label: prawo cywilne, classified: UNKNOWN | CORRECT?: False
OrderedDict([('tax_law', 1)])
5/2721 label: prawo cywilne, classified: prawo podatkowe | CORRECT?: False
OrderedDict([('medical_law', 1)])
6/2721 label: prawo administracyjne, classified: UNKNOWN | CORRECT?: False
OrderedDict([('tax_law', 1)])
7/2721 label: prawo cywilne, classified: prawo podatkowe | CORRECT?: False
OrderedDict([('pharmaceutical_law', 1)])
8/2721 label: prawo cywilne, classified: prawo farmaceutyczne | CORRECT?: False
9/2721 label: prawo cywilne, classified: UNKNOWN | CORRECT?: False
10/2721 label: prawo cywilne, classified: UNKNOWN

In [177]:
d= classifyText("czy wspólnota mieszkaniowa może podjąć uchwałę dotyczącą zakupu wodomierzy odczytywanych radiowo do lokali pokrywając ich koszty czy wspólnota mieszkaniowa może odzyskać pomieszczenie gospodarcze jakie kroki wspólnota mieszkaniowa powinna podjąć aby odzyskać przedmiotowe pomieszczenie czy uprawnienie mieszkańców wspólnoty mieszkaniowej jest zgodne z przepisami rodo czy budowa windy dla osób niepełnosprawnych może zostać zakwalifikowana jako remont i zostać sfinansowana z funduszu remontowego dużej wspólnoty mieszkaniowej jakie czynności powinna podjąć wspólnota mieszkaniowa założenie funduszu celowego w celu wybudowania windy dla osób niepełnosprawnych czy wspólnota mieszkaniowa może uchwałą wspólników przeksięgować środki z funduszu remontowego na fundusz celowy czy nieruchomość nabywa wspólnota mieszkaniowa czy też jej poszczególni członkowie kto będzie właścicielem nabytej nieruchomości kto będzie zobowiązany do zapłaty podatku od nieruchomości czy na zakup nieruchomości wspólnota może przeznaczyć środki zgromadzone na zaliczce remontowo inwestycyjnej czy do nabycia nieruchomości wystarczy podjęcie uchwały większością głosów jaką treść powinny zawierać przedmiotowe tablice informacyjne jakie koszty musi ponieść wspólnota mieszkaniowa w związku ze złożeniem wniosku o stwierdzenie nabycia spadku w którego skład wchodzi nieruchomość pozostająca we wspólnocie czy z uwagi na uchybienie formalne podejmowanie uchwały poprzez indywidualne zbieranie głosów od członków wspólnoty mieszkaniowej które zostało dokonane przez inny podmiot niż zarząd lub zarządcę nieruchomości wspólnej istnieje przesłanka do stwierdzenia jej nieważności czy pomimo tego iż uchwała została podjęta większością głosów to nadal istnieje obowiązek dalszego zbierania głosów czy w treści uchwały należy zamieszczać rubrykę informującą o wstrzymaniu się od głosu w jaki sposób odpowiedzieć na zarzut iż sposób zbierania głosów mógł mieć istotny wpływ na treść uchwały czy zarząd wspólnoty mieszkaniowej ma możliwość wyjścia z inicjatywą anulowania uchwały a następnie zbierania głosów pod identyczną uchwałą z zachowaniem wymogów formalnych czy wspólnota może rozłożyć właścicielom lokali płatność na raty czy z każdym właścicielem lokalu należy zawrzeć odrębną umowę czy wspólnota")
print(d)

OrderedDict([('criminal_law', 2), ('tax_law', 2), ('pharmaceutical_law', 1), ('medical_law', 1)])
criminal_law


In [181]:
doc = nlp1("czy wspólnota mieszkaniowa może podjąć uchwałę dotyczącą zakupu wodomierzy odczytywanych radiowo do lokali pokrywając ich koszty czy wspólnota mieszkaniowa może odzyskać pomieszczenie gospodarcze jakie kroki wspólnota mieszkaniowa powinna podjąć aby odzyskać przedmiotowe pomieszczenie czy uprawnienie mieszkańców wspólnoty mieszkaniowej jest zgodne z przepisami rodo czy budowa windy dla osób niepełnosprawnych może zostać zakwalifikowana jako remont i zostać sfinansowana z funduszu remontowego dużej wspólnoty mieszkaniowej jakie czynności powinna podjąć wspólnota mieszkaniowa założenie funduszu celowego w celu wybudowania windy dla osób niepełnosprawnych czy wspólnota mieszkaniowa może uchwałą wspólników przeksięgować środki z funduszu remontowego na fundusz celowy czy nieruchomość nabywa wspólnota mieszkaniowa czy też jej poszczególni członkowie kto będzie właścicielem nabytej nieruchomości kto będzie zobowiązany do zapłaty podatku od nieruchomości czy na zakup nieruchomości wspólnota może przeznaczyć środki zgromadzone na zaliczce remontowo inwestycyjnej czy do nabycia nieruchomości wystarczy podjęcie uchwały większością głosów jaką treść powinny zawierać przedmiotowe tablice informacyjne jakie koszty musi ponieść wspólnota mieszkaniowa w związku ze złożeniem wniosku o stwierdzenie nabycia spadku w którego skład wchodzi nieruchomość pozostająca we wspólnocie czy z uwagi na uchybienie formalne podejmowanie uchwały poprzez indywidualne zbieranie głosów od członków wspólnoty mieszkaniowej które zostało dokonane przez inny podmiot niż zarząd lub zarządcę nieruchomości wspólnej istnieje przesłanka do stwierdzenia jej nieważności czy pomimo tego iż uchwała została podjęta większością głosów to nadal istnieje obowiązek dalszego zbierania głosów czy w treści uchwały należy zamieszczać rubrykę informującą o wstrzymaniu się od głosu w jaki sposób odpowiedzieć na zarzut iż sposób zbierania głosów mógł mieć istotny wpływ na treść uchwały czy zarząd wspólnoty mieszkaniowej ma możliwość wyjścia z inicjatywą anulowania uchwały a następnie zbierania głosów pod identyczną uchwałą z zachowaniem wymogów formalnych czy wspólnota może rozłożyć właścicielom lokali płatność na raty czy z każdym właścicielem lokalu należy zawrzeć odrębną umowę czy wspólnota")

In [182]:
spacy.displacy.render(doc, style="ent", jupyter=True)